In [1]:
cd ..

/Users/pvm/Documents/Fsoft/mlops-demo


In [2]:

pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv("conf/dev.env")

True

In [4]:
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

from src.modeling.custom.model import CatboostCustomClassifier
from src.modeling.custom.score import MLScorer
from src.pipelines.on_premise_pipeline import Trainer

from src.data_processing.load_data import DataLoader
from src.data_processing.process_data import ChurnDataTransformer
from src.utils.ml_utils import generate_params

# Prepare data

- Bước này ta tiến hành load data (có thể là raw data, sau đó tiến hành transform)

In [5]:
location_raw_data_path = "datasets/raw_datas/WA_Fn-UseC_-Telco-Customer-Churn.csv"
dataloader = DataLoader(location_raw_data_path)
df_data = dataloader.load_data()

In [6]:
data_transformer = ChurnDataTransformer(df_data)

In [7]:
df_tranformed = data_transformer.transform(df_data)

In [8]:
# Prepre dataset
y = df_tranformed[["Churn"]]
X = df_tranformed.drop(columns = ["Churn"])

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 123)


# Setup model Model

In [9]:
custom_scorer = MLScorer()
trainer = Trainer()

# Experiments

In [11]:
grid_values = {'iterations': [200, 300], 'learning_rate':[0.001,.009], 'depth': [5, 10]}

for params in tqdm(generate_params(grid_values)):
    model_catboost = CatboostCustomClassifier(model_params = params)
    model_trained = trainer.train(
                    model = model_catboost, 
                    scorer = custom_scorer, 
                    x_train = X_train, 
                    y_train =y_train, 
                    x_val=X_val, 
                    y_val=y_val, 
                    params=params, 
                    experiment_name="catboost-model"
                )

0it [00:00, ?it/s]2023/04/07 00:04:53 INFO mlflow.tracking.fluent: Experiment with name 'catboost-model' does not exist. Creating a new experiment.
8it [00:17,  2.16s/it]


# Evaluate Model (scoring model)